Run all commands listed in requirements.txt, proper format wasn't working from pipreqs

Also run the commented out nltk import & downloads exactly once to use newspaper's nlp features

In [266]:
import newspaper
from nela_features.nela_features import NELAFeatureExtractor
import numpy as np
import pandas as pd
import jellyfish
import tldextract
import pickle
# Run the below once
#import nltk
#nltk.download('punkt_tab')
#nltk.download('punkt')

In [267]:
def get_article_text(url):
    return newspaper.article(url).text

def smog_to_text(smog):
    if smog >= 17:
        return "Graduate"
    if smog >= 13:
        return "Undergraduate"
    if smog >= 9:
        return "High School"
    if smog >= 5:
        return "Middle School"
    else:
        return "Elementary School"
    
def get_nela_smog(url):
    text = get_article_text(url)
    nela = NELAFeatureExtractor()
    complexity_vector, complexity_names = nela.extract_complexity(text) 
    return smog_to_text(complexity_vector[4])

In [268]:
# using allsides ratings dataset found at https://www.kaggle.com/datasets/supratimhaldar/allsides-ratings-of-bias-in-electronic-media
def get_allsides(url):
    allsides = pd.read_csv('dataset/allsides.csv')
    parsed = tldextract.extract(url)
    website = parsed.domain
    allsides_vals = allsides.values
    sources = allsides_vals[:,0]
    rows = [row for row in allsides_vals if website.lower() in row[0].lower()]
    if rows == []:
        rows = [row for row in allsides_vals if website.lower()[:3] in row[0].lower() and website.lower()[:3] != 'the']
    if rows != []:
        rows.sort(key = lambda x: x[2],reverse=True)
        return rows[0]
    #distances = [jellyfish.levenshtein_distance(website.lower(), x.lower()) for x in sources]
    #loc = distances.index(min(distances))
    distances = [jellyfish.jaro_similarity(website.lower(), x.lower()) for x in sources]
    loc = distances.index(max(distances))
    source_allsides_format = sources[loc]
    row = allsides_vals[loc]
    return row
def get_knn_class(url):
    nela = NELAFeatureExtractor()
    with open('knnfakenews.pkl', 'rb') as f:
        knn = pickle.load(f)
    text = get_article_text(url)
    feature_vector, feature_names = nela.extract_all(text)
    vector = [[feature_vector[i] for i in [24, 22, 4, 86]]]
    return True if knn.predict(vector) == [1] else False

In [269]:
print(get_allsides("https://www.nytimes.com/2024/11/26/world/middleeast/israel-oct-7-inquiry.html"))

0.6825396825396824
['New York Times (News)' 'left-center' 67516 28834 38682
 0.7454113024145598 'Somewhat Disagrees'
 'https://www.allsides.com/news-source/new-york-times']


In [270]:
url = "https://www.nytimes.com/2024/11/26/world/middleeast/israel-oct-7-inquiry.html"
get_knn_class(url)
print(get_allsides(url))

0.6825396825396824
['New York Times (News)' 'left-center' 67516 28834 38682
 0.7454113024145598 'Somewhat Disagrees'
 'https://www.allsides.com/news-source/new-york-times']


In [271]:
url = "https://theonion.com/announcement-of-fourth-child-contains-conspicuous-lack-of-exclamation-points/"
get_knn_class(url)
get_nela_smog(url)
print(get_allsides(url))

0.8583333333333334
['The Nation' 'left' 2091 1599 492 3.25 'Absolutely Agrees'
 'https://www.allsides.com/news-source/nation-media-bias']


In [272]:
url = "https://www.cbsnews.com/news/tom-homan-greg-abbott-texas-border-visit/"
get_knn_class(url)
print(get_allsides(url))

['CBS News (Online)' 'left-center' 30930 18873 12057 1.5653147549141575
 'Agrees' 'https://www.allsides.com/news-source/cbs-news-media-bias']


In [273]:
url = "https://skepticalinquirer.org/exclusive/are-saunas-good-for-you-yes-but/"
get_knn_class(url)
print(get_allsides(url))

0.698529411764706
['Splinter' 'left' 381 257 124 2.0725806451612905 'Strongly Agrees'
 'https://www.allsides.com/news-source/splinter-media-bias']


In [274]:
url = 'https://jacobin.com/2024/11/trump-biden-israel-middle-east'
get_knn_class(url)
print(get_allsides(url))

['Jacobin' 'left' 1210 974 236 4.127118644067797 'Absolutely Agrees'
 'https://www.allsides.com/news-source/jacobin-media-bias']


In [275]:
url = 'https://www.cnn.com/2023/10/24/politics/supreme-court-florida-anti-drag-law/index.html'
get_knn_class(url)
print(get_allsides(url))

['CNN (Online News)' 'left' 98448 50713 47735 1.0623860898711637
 'Somewhat Agrees' 'https://www.allsides.com/news-source/cnn-media-bias']


In [276]:
url = 'https://www.forbes.com/sites/alexkonrad/2024/11/27/elon-musk-doge-attracts-young-coders-and-tech-ceos/'
get_knn_class(url)
print(get_allsides(url))

['Forbes' 'center' 21999 13638 8361 1.6311445999282383 'Agrees'
 'https://www.allsides.com/news-source/forbes']
